# Домашнее задание «Деревья решений»

In [65]:
# Подключение необходимых библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import warnings

warnings.filterwarnings('ignore')

Загрузим датасет

In [66]:
from sklearn.datasets import load_boston

boston = load_boston()
data = pd.DataFrame(boston['data'], columns=boston['feature_names'])
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [67]:
y = pd.DataFrame(boston['target'], columns=['MEDV'])
y.head()

,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


Создадим функции для оценки качества моделей. Оценивать работу модели будем по тестовой выборке (будем делить выборку на train и test)

In [77]:
# Оценка линейной регресии
def get_score_LR(X, y):
    model = LinearRegression()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42 )  
    model.fit(X_train, y_train)
    
    return model.score(X_test, y_test)

# Оценка дерева решений
def get_score_DTR(X, y, max_depth=None, min_samples_leaf=1):
    if max_depth is None:
        model = DecisionTreeRegressor(min_samples_leaf=min_samples_leaf)
    
    else:
        model = DecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=min_samples_leaf)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42 )  
    model.fit(X_train, y_train)
    
    return model.score(X_test, y_test)

Немного преобразуем данные для лучшего качества работы модели

In [69]:
# Нормализация данных
columns = ['ZN', 'INDUS', 'NOX', 'AGE', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT']

for col in columns:
    data[col] -= data[col].mean()
    data[col] /= data[col].std()

In [70]:
# Округление количество комнат до целого
data['RM'] = data['RM'].apply(lambda x: round(x))
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,0.284548,-1.286636,0.0,-0.144075,7,-0.119895,4.0900,-0.981871,-0.665949,-1.457558,0.440616,-1.074499
1,0.02731,-0.487240,-0.592794,0.0,-0.739530,6,0.366803,4.9671,-0.867024,-0.986353,-0.302794,0.440616,-0.491953
2,0.02729,-0.487240,-0.592794,0.0,-0.739530,7,-0.265549,4.9671,-0.867024,-0.986353,-0.302794,0.396035,-1.207532
3,0.03237,-0.487240,-1.305586,0.0,-0.834458,7,-0.809088,6.0622,-0.752178,-1.105022,0.112920,0.415751,-1.360171
4,0.06905,-0.487240,-1.305586,0.0,-0.834458,7,-0.510674,6.0622,-0.752178,-1.105022,0.112920,0.440616,-1.025487


Посчитаем начальное качество работы моделей

In [71]:
# Линейная регрессия
get_score_LR(data, y)

0.7248493104687765

In [74]:
# Дерево решений
get_score_DTR(data, y)

0.8022521743421813

Качество на дереве решений получилось существенно выше

Поиграемся с параметрами глубина дерева и количество семплов в листе

In [78]:
# Переберем глубину от 3 до 10 и посчитаем в каждом случае качество
for level in range(3,11):
    print("Глубина", level, "качество -", get_score_DTR(data, y, max_depth=level))

Глубина 3 качество - 0.7349276293208288
Глубина 4 качество - 0.7756021720615456
Глубина 5 качество - 0.7866409325241195
Глубина 6 качество - 0.8081977340248209
Глубина 7 качество - 0.8011679286874325
Глубина 8 качество - 0.7664749663009407
Глубина 9 качество - 0.7894342014744218
Глубина 10 качество - 0.8075813341306635


In [81]:
# Берем количество семплов в листе от 1 до 5 и посчитаем в каждом случае качество
for leaf in range(1,6):
    print("Количество семплов в листе", leaf, "качество -", get_score_DTR(data, y, None, leaf))

Количество семплов в листе 1 качество - 0.7859030524203715
Количество семплов в листе 2 качество - 0.6883169592571633
Количество семплов в листе 3 качество - 0.7675320543591819
Количество семплов в листе 4 качество - 0.7303249722211452
Количество семплов в листе 5 качество - 0.7539993326811109


Из перебора видно, что лучшее качество получается при глубине 6 и размере листа 1.\
Построим модель с этими параметрами и посмотрим на ее качество

In [82]:
get_score_DTR(data, y, max_depth=6, min_samples_leaf=1)

0.8096643805358875

Качество стало выше чем изначальное